In [182]:
import os
import warnings
import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, brier_score_loss, f1_score
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [183]:
print("making train_dataset")
X_train = pd.read_csv('./soccer_multiclass_data/X_train',index_col=0)
Y_train = pd.read_csv("./soccer_multiclass_data/Y_train",index_col=0)            

making train_dataset


predict

In [184]:
for col in X_train.columns:
    if col.startswith('body'):
        print(col)

bodypart_id_a0
bodypart_id_a1
bodypart_id_a2
bodypart_foot_a0
bodypart_head_a0
bodypart_other_a0
bodypart_head/other_a0
bodypart_foot_a1
bodypart_head_a1
bodypart_other_a1
bodypart_head/other_a1
bodypart_foot_a2
bodypart_head_a2
bodypart_other_a2
bodypart_head/other_a2


In [185]:
X_train

,type_id_a0,type_id_a1,type_id_a2,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,...,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,0,0,0,True,False,False,False,False,False,False,...,60.979337,0.016940,60.979337,0.016940,60.979337,0.016940,True,True,0.0,0.0
1,0,0,0,True,False,False,False,False,False,False,...,68.399682,0.184780,60.979337,0.016940,60.979337,0.016940,True,True,5.0,5.0
2,21,0,0,False,False,False,False,False,False,False,...,83.032777,0.172945,39.800813,0.321247,44.041526,0.023455,False,False,4.0,9.0
3,0,21,0,True,False,False,False,False,False,False,...,86.482277,0.150862,83.032777,0.172945,39.800813,0.321247,True,False,1.0,5.0
4,21,0,21,False,False,False,False,False,False,False,...,88.427926,0.356986,86.482277,0.150862,83.032777,0.172945,True,True,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998775,21,0,0,False,False,False,False,False,False,False,...,77.110477,0.372455,77.110477,0.372455,93.483973,0.133909,True,False,4.0,5.0
998776,0,21,0,True,False,False,False,False,False,False,...,34.787188,0.893744,77.110477,0.372455,77.110477,0.372455,True,True,0.0,4.0
998777,21,0,21,False,False,False,False,False,False,False,...,21.893732,0.425538,34.787188,0.893744,77.110477,0.372455,True,True,5.0,5.0
998778,7,21,0,False,False,False,False,False,False,False,...,21.893732,0.425538,21.893732,0.425538,34.787188,0.893744,True,True,4.0,9.0


In [186]:
print("making valid_dataset")
X_valid = pd.read_csv('./soccer_multiclass_data/X_valid',index_col=0)
Y_valid = pd.read_csv("./soccer_multiclass_data/Y_valid",index_col=0)          

making valid_dataset


In [187]:
print("making test_dataset")
X_test = pd.read_csv('./soccer_multiclass_data/X_test',index_col=0)
Y_test = pd.read_csv("./soccer_multiclass_data/Y_test",index_col=0)   

making test_dataset


In [188]:
X_train.shape, Y_train.shape,X_valid.shape, Y_valid.shape,X_test.shape, Y_test.shape

((998780, 154),
 (998780, 1),
 (75786, 154),
 (75786, 1),
 (82104, 154),
 (82104, 1))

In [189]:
time_col = ['time_seconds_a0','time_seconds_overall_a0',
            'time_seconds_a1','time_seconds_overall_a1',
            'time_seconds_a2','time_seconds_overall_a2',
            'period_id_a0','period_id_a1','period_id_a2']

In [190]:
# X_train.drop(columns=time_col,inplace=True)
# X_valid.drop(columns=time_col,inplace=True)
# X_test.drop(columns=time_col,inplace=True)

KeyError: "['time_seconds_a0', 'time_seconds_overall_a0', 'time_seconds_a1', 'time_seconds_overall_a1', 'time_seconds_a2', 'time_seconds_overall_a2', 'period_id_a0', 'period_id_a1', 'period_id_a2'] not found in axis"

In [ ]:
drop_list = []
for col in X_train.columns:
    if col.startswith("time") or col.startswith("period"):
        drop_list.append(col)
drop_list      

['time_delta_1', 'time_delta_2']

In [ ]:
# X_train.drop(columns=drop_list,inplace=True)
# X_valid.drop(columns=drop_list,inplace=True)
# X_test.drop(columns=drop_list,inplace=True)

In [191]:
X_train.shape, Y_train.shape,X_valid.shape, Y_valid.shape,X_test.shape, Y_test.shape

((998780, 154),
 (998780, 1),
 (75786, 154),
 (75786, 1),
 (82104, 154),
 (82104, 1))

### 여러 머신러닝 기법을 적용하여 다중 클래스 분류 학습

#### train데이터는 매 학습마다 섞어줘야 할 것 같음

In [201]:
import random

# 데이터의 인덱스를 생성
indices = list(range(len(X_train)))

# 인덱스를 섞음
random.shuffle(indices)

# 섞인 인덱스를 사용하여 X_train과 Y_train을 섞음
X_train_shuffled  = [X_train.loc[i] for i in indices]
Y_train_shuffled  = [Y_train.loc[i] for i in indices]


In [199]:
X_train_shuffled_train = pd.DataFrame(X_train_shuffled)
Y_train_shuffled_train = pd.DataFrame(Y_train_shuffled)

In [202]:
X_train_shuffled_train

,type_id_a0,type_id_a1,type_id_a2,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,...,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
38513,22,11,4,False,False,False,False,False,False,False,...,76.176770,0.437561,105.091727,0.041784,91.594595,0.116794,False,False,24.0,26.0
783166,21,19,21,False,False,False,False,False,False,False,...,57.010983,0.019629,53.799057,0.094538,53.799057,0.094538,False,False,0.0,1.0
710752,21,0,0,False,False,False,False,False,False,False,...,58.905507,0.350529,68.040170,0.261030,56.627958,0.242535,False,True,2.0,2.0
154044,21,0,21,False,False,False,False,False,False,False,...,64.545072,0.395646,62.268372,0.476798,63.105891,0.310499,True,True,1.0,2.0
540041,0,21,0,True,False,False,False,False,False,False,...,73.143504,0.475495,66.353082,0.359229,66.353082,0.359229,True,True,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954883,0,21,0,True,False,False,False,False,False,False,...,54.903761,0.428445,63.107092,0.391863,62.932610,0.297047,True,True,1.0,2.0
416840,21,0,21,False,False,False,False,False,False,False,...,21.712278,0.464500,22.797790,0.521732,36.212100,0.500829,True,True,1.0,3.0
6082,0,21,0,True,False,False,False,False,False,False,...,61.848337,0.453811,61.454260,0.276642,61.105104,0.307693,True,True,1.0,2.0
355639,21,0,21,False,False,False,False,False,False,False,...,59.090188,0.436135,59.090188,0.436135,61.748085,0.520891,True,True,1.0,1.0


In [203]:
Y_train_shuffled_train

,label
38513,0
783166,0
710752,0
154044,0
540041,0
...,...
954883,0
416840,0
6082,0
355639,0


In [204]:
# 3. train classifiers F(X) = Y
from sklearn.model_selection import GridSearchCV


from tqdm import tqdm

import catboost
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
# from pytorch_tabnet.tab_model import TabNetClassifier
Y_hat = pd.DataFrame()
models = {}

for col in tqdm(list(Y_train.columns)):
    model = catboost.CatBoostClassifier()
    #model = xgboost.XGBClassifier()
    #model = RandomForestClassifier()
    #model = MLPClassifier()
    #model = svm.SVC()
    #model = LogisticRegression(solver='lbfgs', max_iter=5000)
    #model = DecisionTreeClassifier()
    #model = TabNetMultiTaskClassifier()
    #model = TabNetClassifier()
    #clf = TabNetClassifier()     
    model.fit(X_train_shuffled,Y_train_shuffled,eval_set=[(X_valid,Y_valid)])
    
    predict = model.predict_proba(X_test)

  0%|          | 0/1 [00:00<?, ?it/s]

Learning rate set to 0.12397
0:	learn: 0.8772135	test: 0.8764346	best: 0.8764346 (0)	total: 309ms	remaining: 5m 8s
1:	learn: 0.7238178	test: 0.7224919	best: 0.7224919 (1)	total: 693ms	remaining: 5m 45s
2:	learn: 0.6096410	test: 0.6078764	best: 0.6078764 (2)	total: 995ms	remaining: 5m 30s
3:	learn: 0.5209848	test: 0.5188125	best: 0.5188125 (3)	total: 1.23s	remaining: 5m 5s
4:	learn: 0.4499814	test: 0.4474404	best: 0.4474404 (4)	total: 1.46s	remaining: 4m 51s
5:	learn: 0.3925473	test: 0.3896530	best: 0.3896530 (5)	total: 1.84s	remaining: 5m 4s
6:	learn: 0.3443429	test: 0.3412510	best: 0.3412510 (6)	total: 2.1s	remaining: 4m 58s
7:	learn: 0.3041650	test: 0.3008889	best: 0.3008889 (7)	total: 2.35s	remaining: 4m 52s
8:	learn: 0.2709504	test: 0.2673787	best: 0.2673787 (8)	total: 2.61s	remaining: 4m 47s
9:	learn: 0.2423529	test: 0.2386482	best: 0.2386482 (9)	total: 2.86s	remaining: 4m 43s
10:	learn: 0.2180636	test: 0.2142073	best: 0.2142073 (10)	total: 3.16s	remaining: 4m 44s
11:	learn: 0.197

100%|██████████| 1/1 [10:18<00:00, 618.43s/it]


### 학습시킨 model를 이용하여 평가하기

1. score인지 평가
2. concede인지 평가

In [ ]:
predict.shape

(82104, 3)

In [ ]:
pd.DataFrame(predict,columns=['Not','score','concede'])

,Not,score,concede
0,0.994866,0.003373,0.001761
1,0.993546,0.005010,0.001444
2,0.995437,0.003189,0.001373
3,0.995914,0.002638,0.001448
4,0.995445,0.002940,0.001615
...,...,...,...
82099,0.982012,0.017230,0.000758
82100,0.990435,0.000794,0.008771
82101,0.989195,0.005525,0.005281
82102,0.992742,0.003607,0.003651


In [ ]:
test_score = (np.array(Y_test) == 1).reshape(-1)
test_concede = (np.array(Y_test) == 2).reshape(-1)

test_score.shape,test_concede.shape

((82104,), (82104,))

In [ ]:
print("roc auc : " ,roc_auc_score(test_score, predict[:,1]))
print("roc auc : " ,roc_auc_score(test_concede, predict[:,2]))

roc auc :  0.8026327441488432
roc auc :  0.8793344975050271


In [ ]:
pd.DataFrame(predict,columns=['Not','score','concede'])[['score','concede']].to_csv("./score concede probabiliy data/catboost probability")

OSError: Cannot save file into a non-existent directory: 'score concede probabiliy data'